This tutorial explains details about existing parsers and how to create a new parser if necessary.

# 2022 Google Android Derived Dataset

This data comes from the 2022 Google Smartphone Decimeter Challenge and can be downloaded from [Kaggle](https://www.kaggle.com/c/smartphone-decimeter-2022).

In [ ]:
from gnss_lib_py import AndroidDerived2022

Loading the data into an instance of `NavData` is as easy as creating an instance of `AndroidDerived2022` with the relevant file path.

In [ ]:
# download Android data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2022/device_gnss.csv --quiet -O "device_gnss.csv"
# load Android Google Challenge data
derived_data = AndroidDerived2022("device_gnss.csv")

We can verify that the data loaded correctly by printing the shape and rows of the imported data.

In [ ]:
derived_data.shape

In [ ]:
derived_data.rows

# 2022 Google Android Ground Truth
We can similarly load in the ground truth data from the same 2022 Google Smartphone Decimeter Challenge.

In [ ]:
from gnss_lib_py import AndroidGroundTruth2022

In [ ]:
# download Android data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2022/ground_truth.csv --quiet -O "ground_truth.csv"
# load Android Google Challenge ground truth data
gt_data = AndroidGroundTruth2022("ground_truth.csv")

In [ ]:
gt_data.shape

In [ ]:
gt_data.rows

# 2022 Google Decimeter Kaggle Challenge
There are utility functions to prepare Kaggle submissions for the 2022 challenge.

We offer a function to convert the provided Weighted Least Squares baseline solution into the standard `state_estimate` format found throughout `gnss_lib_py`. Simply pass in the derived data NavData object.

In [ ]:
from gnss_lib_py import solve_kaggle_baseline

state_estimate = solve_kaggle_baseline(derived_data)

print(state_estimate)

`prepare_kaggle_submission` can be used to convert the standard `state_estimate` format to a NavData object with the same rows and row names which the 2022 Kaggle competition expects. The `trip_id` is a combination of the trajectory trace name and phone name.

In [ ]:
from gnss_lib_py import prepare_kaggle_submission

solution = prepare_kaggle_submission(state_estimate, trip_id = "my_trace/my_phone")

print(solution)

`solve_kaggle_dataset` can be used to automatically iterate through all trace trajectory names and phone names, estimate the state using the provided solver, and concatenate all state estimates together for a single submission. The `solver` variable could use `solve_kaggle_baseline`, `solve_wls`, or `solve_gnss_ekf` for example.


In [ ]:
from gnss_lib_py import solve_kaggle_dataset
from gnss_lib_py import solve_wls

# submission = solve_kaggle_dataset(folder_path = "/path/to/train/or/test/folder/", 
#                                   solver = solve_wls)

# after the submission NavData object is created, save it to a csv with:
# submission.to_csv("file_path.csv")

# 2021 Google Android Derived Dataset

This data comes from the 2021 Google Smartphone Decimeter Challenge and can be downloaded from [Kaggle](https://www.kaggle.com/c/google-smartphone-decimeter-challenge).

In [ ]:
from gnss_lib_py import AndroidDerived2021

Loading the data into an instance of `NavData` is as easy as creating an instance of `AndroidDerived2021` with the relevant file path.

In [ ]:
# download Android data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4_derived.csv --quiet -O "Pixel4_derived.csv"
# load Android Google Challenge data
derived_data = AndroidDerived2021("Pixel4_derived.csv")

We can verify that the data loaded correctly by printing the shape and rows of the imported data.

In [ ]:
derived_data.shape

In [ ]:
derived_data.rows

# 2021 Google Android Ground Truth
We can similarly load in the ground truth data from the same 2021 Google Smartphone Decimeter Challenge.

In [ ]:
from gnss_lib_py import AndroidGroundTruth2021

In [ ]:
# download Android data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4_ground_truth.csv --quiet -O "Pixel4_ground_truth.csv"
# load Android Google Challenge ground truth data
gt_data = AndroidGroundTruth2021("Pixel4_ground_truth.csv")

In [ ]:
gt_data.shape

In [ ]:
gt_data.rows

# Precise Ephemerides Processing

The data required to calculate with precise ephemerides uses .sp3 and .clk files, which can be downloaded from [CDDIS](https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/gnss_mgex.html) or [CORS](https://geodesy.noaa.gov/UFCORS/).

The .sp3 files provide post-processed, accurate, and precise information regarding 3-D satellite position in the Earth-Centered Earth-Fixed (ECEF) frame at intervals of 15mins each.  

Similarly, the .clk files provide post-processed, accurate and precise information on satellite clock errors at intervals of 30secs each.  

These .sp3 and .clk files are available for any GNSS constellation, and hence, provide a common processing platform for applications that involve multi-GNSS satellite signals (without requiring to parse the broadcast ephemeris from each constellation separately one at a time). Also, unlike broadcast ephemeris that can suffer from signal-in-space anomalies, the .sp3 and .clk files are guaranteed to provide accurate satellite information. However, note that, these files are only available in a post-processed manner, and not in real-time

We show how to analyze this precise ephemerides functionality for the Android derived dataset in the following cells, 

1. Load the derived data from AndroidDerived

In [ ]:
from gnss_lib_py import multi_gnss_from_precise_eph
from gnss_lib_py import AndroidDerived2021
import numpy as np

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4_derived_clkdiscnt.csv --quiet -O "Pixel4_derived_clkdiscnt.csv"
derived_data = AndroidDerived2021("Pixel4_derived_clkdiscnt.csv", remove_timing_outliers=False)

2. Specify the paths to the .sp3 and .clk files, as well as the desired constellations from which one intends to extract satellite information

In [ ]:
# download .sp3 data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/precise_ephemeris/grg21553_short.sp3 --quiet -O "grg21553_short.sp3"
# Specify .sp3 file path to extract precise ephemerides
sp3_path = "grg21553_short.sp3"

# download .clk data file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/precise_ephemeris/grg21553_short.clk --quiet -O "grg21553_short.clk"
# Specify .clk file path to extract precise ephemerides
clk_path = "grg21553_short.clk"

3. Remove the rows in NavData class that refer to satellite information (3-D satellite position, 3-D satellite velocity, clock bias and clock drift), 

    i.e., `SV_KEYS =['x_sv_m', 'y_sv_m', 'z_sv_m', 'vx_sv_mps','vy_sv_mps','vz_sv_mps', 'b_sv_m', 'b_dot_sv_mps']`

In [ ]:
# Define the keys relevant for satellite information, and remove the data within these fields
SV_KEYS = ['x_sv_m', 'y_sv_m', 'z_sv_m', \
           'vx_sv_mps','vy_sv_mps','vz_sv_mps', \
           'b_sv_m', 'b_dot_sv_mps']
derived_data[SV_KEYS] = np.nan

4. Populate the columns of SV_KEYS with information extracted via precise ephemerides

In [ ]:
# Update derived_data class with satellite information computed via precise ephemerides
derived_multi_gnss = multi_gnss_from_precise_eph(derived_data, sp3_path, \
                                                 clk_path, verbose = False)

In [ ]:
# Check that all the desired fields related to satellite information have useful
# information, and norm of computed satellite position matches the altitude of GNSS constellation
derived_multi_gnss.copy(rows=["gnss_id","sv_id"] + SV_KEYS)

In [ ]:
# Check that all the desired fields related to satellite information have useful
# information, and norm of computed satellite position matches the altitude of GNSS constellation
sat_alt = np.linalg.norm(derived_multi_gnss[['x_sv_m', 'y_sv_m', 'z_sv_m'],5:7],axis=0)
print('Distance of two satellites from the center of the Earth (expected around 26000000 m):')
print(derived_multi_gnss["gnss_id"][5:7], sat_alt)
print("Extracted values for same satellites: ")
for sv in SV_KEYS:
    print(sv, ':', derived_multi_gnss[sv][5:7])

# How to Create a New NavData Class

The modular and versatile functionality of this gnss_lib_py repository is enabled by loading all data types into a custom Python NavData class. If you are using a type of data or dataset that is not yet supported, you will need to create a new child class of the NavData Python class. This tutorial will guide you on how to set up your new Python child class. Once complete, please feel free to submit a pull request to our GitHub repository so other users can  make use of the added functionality.

For this example, say that we have a new type of data called MyReceiver that is a csv file with columns of a timestamp, satellite identifier, and pseudorange. The contents of a sample `myreceiver.csv` is the following:

| myTimestamp | mySatId | myPseudorange |
| ----------- | ------- | ------------- |
| 10          | 10      | 270000001     |
| 10          | 14      | 270000007     |
| 10          | 7       | 270000004     |
| 10          | 3       | 270000005     |
| 11          | 10      | 270000002     |
| 11          | 14      | 270000008     |
| 11          | 7       | 270000003     |
| 11          | 3       | 270000004     |

The first step is importing the base `NavData` class and creating a new class type that inherits from `NavData`

```python
class MyReceiver(NavData):
    """Class handling measurements from MyReceiver.

    Inherits from NavData().
    """
```

The `__init__` function should have a call to the parent `NavData` `__init__` function. Based on your data input, you should call the corresponding initializer.

For csv files, call: `super().__init__(csv_path=input_path)`  
For pandas DataFrames, call `super().__init__(pandas_df=input_path)`  
For numpy ndarrays, call `super().__init__(numpy_array=input_path)`  

In our case, we have a csv file, so our `__init__` function looks like the following:

```python
def __init__(self, input_path):
    """MyReceive specific loading and preprocessing

    Parameters
    ----------
    input_path : string
        Path to MyReceiver csv file

    """

    # call NavData initialization with csv path
    super().__init__(csv_path=input_path)
```

After our data is loaded, we may want to make known changes to our data. We can make those changes by defining a `postprocess` function. NavData's `__init__` function that we call in our initializer already makes a call to the `postprocess` function, so we don't have to repeat that call in `MyReceiver`'s `__init__` function.

One thing that we need to do to make use of the common functionality of `gnss_lib_py` is to standardize the names of our variables. See the [Standard Naming Conventions](https://gnss-lib-py.readthedocs.io/en/latest/reference/reference.html#standard-naming-conventions) section in the Reference tab of the documentation for the list of standardized names.

In our case, we will convert `mySatId` to `sv_id` and `myPseudorange` to `raw_pr_m`. We make these conversions by simply updating the `_row_map` function.

```python


@staticmethod
def _row_map():
    """Map of column names from loaded to gnss_lib_py standard

    Returns
    -------
    row_map : Dict
        Dictionary of the form {old_name : new_name}
    """
    row_map = {'mySatId' : 'sv_id',
               'myPseudorange' : 'raw_pr_m',
               }
    return row_map
```

As an additional postprocessing step, we may want to offset our pseudorange due to a known error or create the common timestamp variable `gps_millis` based on our unique timestamp row. Adding the `gps_millis` row enables the use of some of the common algorithms. The [time conversion utilities](https://gnss-lib-py.readthedocs.io/en/latest/tutorials/tutorials_utilities_notebook.html) can be used to create `gps_millis` from the GPS Week & Time of week, GPS milliseconds, or a datetime object.

```python
# correct pseudorange
self['corr_pr_m'] = self['raw_pr_m'] + 100.

# create common timestamp
self['gps_millis'] = self['myTimestamp'] + 5629719023
```

Putting it all together, we have:

In [ ]:
from gnss_lib_py import NavData

class MyReceiver(NavData):
    """Class handling measurements from MyReceiver.

    Inherits from NavData().
    """
    def __init__(self, input_path):
        """MyReceive specific loading and preprocessing

        Parameters
        ----------
        input_path : string
            Path to MyReceiver csv file

        """
        
        # call NavData initialization with csv path
        super().__init__(csv_path=input_path)
        
    def postprocess(self):
        """MyReceiver specific postprocessing

        """

        # correct pseudorange
        self['corr_pr_m'] = self['raw_pr_m'] + 100.
        
        # create common timestamp
        self['gps_millis'] = self['myTimestamp'] + 1659075505350
        

    @staticmethod
    def _row_map():
        """Map of column names from loaded to gnss_lib_py standard

        Returns
        -------
        row_map : Dict
            Dictionary of the form {old_name : new_name}
        """
        row_map = {'mySatId' : 'sv_id',
                   'myPseudorange' : 'raw_pr_m',
                   }
        return row_map

We can now create a instance of our new `MyReceiver` class with the path to our csv called `myreceiver.csv`.

In [ ]:
# download myreceiver.csv file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/notebooks/tutorials/myreceiver.csv --quiet -O "myreceiver.csv"

# create instance of MyReceiver
my_receiver_data = MyReceiver("myreceiver.csv")

Let's print out our corrected pseudorange to make sure everything worked correctly.

In [ ]:
my_receiver_data["corr_pr_m"]

We can now take advantage of all the tools `gnss_lib_py` has to offer!

In [ ]:
from gnss_lib_py import plot_metric
fig = plot_metric(my_receiver_data,"gps_millis","corr_pr_m",groupby="sv_id")